# Data Cleaning + EDA Process

In [1]:

#~ importing all packages
import requests, pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from pathlib import Path



# ~ indicating where the datasets are stored

#DATA_FOLDER = Path('C:/Users/calvi/INSY 662_Python/datasets')  #! replace with your correct path!!! 

## BIXI Station Status Dataset

In [2]:
''' This block of code will serve to import and read BIXI's station information dataset found as a json file
in BIXI's official site looking at their GBSF feed '''

URL = "https://gbfs.velobixi.com/gbfs/2-2/en/station_information.json"

raw = requests.get(URL, timeout=30).json()

#~ transforming the json file into a dataframe

stations_df = pd.json_normalize(raw["data"]["stations"])

#~ selecting all columns from the json file

keep = [
    "station_id", "name", "short_name", "lat", "lon", "capacity",
    "address", "post_code", "region_id", "external_id",
    "rental_methods"
]
stations_df = stations_df[[c for c in keep if c in stations_df.columns]].copy()


#~ checks if rental_methods column is a list, if it is a list, 
#~ then join (e.g. ["CREDITCARD","APPLEPAY"]), turn it into "CREDITCARD,APPLEPAY".)

if "rental_methods" in stations_df.columns:
    stations_df["rental_methods"] = stations_df["rental_methods"].apply(
        lambda x: ",".join(x) if isinstance(x, list) else x
    )

#~ viewing stations_df
stations_df



ConnectionError: HTTPSConnectionPool(host='gbfs.velobixi.com', port=443): Max retries exceeded with url: /gbfs/2-2/en/station_information.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1513b0950>: Failed to resolve 'gbfs.velobixi.com' ([Errno 8] nodename nor servname provided, or not known)"))

### Understand the dataset - BIXI Station Dataset

In [ ]:

#~ see the numbers of rows and columns 
stations_df.shape

In [ ]:

#~quick data type check and non-null count for stations_df

stations_df.info()

In [ ]:

#~ preview of dataframe head and tail

stations_df.head()

In [ ]:
stations_df.tail()

In [ ]:

#~ list of all columns names

stations_df.columns.tolist()


### Data Quality Check (Data Cleaning Stage) - BIXI Station Dataset

In [ ]:

#~ quick data type changes and dropping unecessary column
stations_df['station_id'] = stations_df['station_id'].astype(int)
stations_df['short_name'] = stations_df['short_name'].astype(int)

stations_df = stations_df.drop(columns=['rental_methods','external_id'])



In [ ]:

#~ checking updated dataframe

''' it seems there is no null value (yay!) '''

stations_df.info()

In [ ]:

#~ checking if duplicated rows exist

stations_df.duplicated().any()

In [ ]:

#~checking if there are duplicates in each columns

"""the duplicates for latitude, longtitude, and capacity make since!   """



column_duplicate_check = stations_df.apply(lambda s: s.nunique(dropna=False)< len(s))

print(column_duplicate_check)

In [ ]:

#~ the dataframe below is the cleaned stations_df

clean_stations_df = stations_df.copy()

clean_stations_df

### Summary statistics (High-Level profile) - BIXI Station Dataset



In [ ]:

#~ high level statistic of all columns

clean_stations_df.describe(include='all')


In [ ]:

#~ frequency of values for capacity

clean_stations_df['capacity'].value_counts()

In [ ]:

#~ proportion of each values within capacity column

clean_stations_df['capacity'].value_counts(normalize=True)*100

### Quick Univariate Analysis - BIXI Station Dataset

In [ ]:

#~ capacity seems to be interesting

sns.boxenplot(x=clean_stations_df['capacity'])


In [ ]:

#~ verifying if certain stations are outliers --> it seems they are very niche cases! Only station #724 that has 3 capacity is weird.

filter_station = (clean_stations_df["capacity"] > 40) | (clean_stations_df["capacity"] < 10)
clean_stations_df.loc[filter_station]

## BIXI 2024 Trip History Dataset

In [ ]:

#~ importing and reading BIXI 2024 Trip history dataset (OVER 13M rows of data!!)
#~ data source: https://bixi.com/en/open-data/   choose the 2024

bixi_trip_df =  pd.read_csv('BIXI_Trip_2024.csv')   
# bixi_trip_df =  pd.read_csv('BIXI_Trip_2024.csv')   

#! I changed the downloaded csv to this name, make sure to reflect if not the same name



### Understand the dataset - BIXI 2024 Trip History Dataset

In [ ]:

#~ looking at shape

bixi_trip_df.shape

In [ ]:

#~ checking data types 
bixi_trip_df.info()

In [ ]:

#~ head and tail of bixi trip dataframe

bixi_trip_df.head()

In [ ]:

bixi_trip_df.tail()


### Data Quality Check - BIXI 2024 Trip History Dataset

In [ ]:

#~ standardizing the columnNames
bixi_trip_df.columns = (bixi_trip_df.columns              
              .str.normalize('NFKD')                    
              .str.encode('ascii','ignore').str.decode('ascii')
              .str.strip()
              .str.lower()
              .str.replace(r'[^a-z0-9]+', '_', regex=True)  
              .str.replace(r'_+', '_', regex=True)          
              .str.strip('_'))

bixi_trip_df.head()

In [ ]:

#~ checking for null values


bixi_trip_df.isnull().sum()



In [ ]:


#~ it seems that a majority of the null values are missing the endstation name and therefore has no endtime. We will assume that this is error in data capture
#~ dropping all null values

bixi_trip_df= bixi_trip_df.dropna()

In [ ]:

#~ double checking to make sure
bixi_trip_df.isnull().sum()

In [ ]:

#~ checking how many rows left after dropping nulls :  went from 13,275,326  to 13,199,617   dropped exactly 75,709 rows!!
bixi_trip_df.shape

In [ ]:

#~ creating a new column by transforming the startimems and endtimems  (these columns are in UNIX format)
#~ into datetime formatted as YYYY-MM-DD HH-mm-ss

bixi_trip_df['starttime'] = (
    pd.to_datetime(bixi_trip_df["starttimems"], unit='ms', utc=True)
      .dt.floor("s")                           #~ the data showed time zone in ISO-8601 format, I stopped at seconds
      .dt.tz_convert("America/Toronto")
      .dt.tz_localize(None)        
)


bixi_trip_df['endtime'] = (
    pd.to_datetime(bixi_trip_df["endtimems"], unit='ms', utc=True)
      .dt.floor("s")                           
      .dt.tz_convert("America/Toronto")
      .dt.tz_localize(None)       
)

bixi_trip_df = bixi_trip_df.drop(columns=['starttimems', 'endtimems']) #~ dropping the initial unix time columns



In [ ]:

#~ there are duplicated rows but this is normal since each row in this dataset represent trip data
bixi_trip_df.duplicated().any()

In [ ]:

#~ checking the dtypes and updating clean df as clean

bixi_trip_df.info()

clean_bixi_trip_df = bixi_trip_df.copy()

In [ ]:

#~ adding column to bring out the hour of the day, the day of week, day of month, and month of the year for starting time and endtime

#~Hour of day (0–23)

clean_bixi_trip_df["start_hour"] = clean_bixi_trip_df["starttime"].dt.hour
clean_bixi_trip_df["end_hour"]   = clean_bixi_trip_df["endtime"].dt.hour

#~Day of week (1–7, Monday=1, Sunday=7)

clean_bixi_trip_df["start_day"] = clean_bixi_trip_df["starttime"].dt.dayofweek + 1
clean_bixi_trip_df["end_day"]   = clean_bixi_trip_df["endtime"].dt.dayofweek + 1

#~Day of month (1–31)
clean_bixi_trip_df["start_dayofmonth"] = clean_bixi_trip_df["starttime"].dt.day
clean_bixi_trip_df["end_dayofmonth"]   = clean_bixi_trip_df["endtime"].dt.day

#~Month of year (1–12)

clean_bixi_trip_df["start_month"] = clean_bixi_trip_df["starttime"].dt.month
clean_bixi_trip_df["end_month"]   = clean_bixi_trip_df["endtime"].dt.month

clean_bixi_trip_df.tail()

### Quick Univariate Analysis - BIXI 2024 Trip History Dataset

In [ ]:

#~ counts of top 10 starting and ending stations

top10_start_count = clean_bixi_trip_df['startstationname'].value_counts().head(10).sort_values()

top10_start_count.plot(kind='barh', xlabel='Count', ylabel=clean_bixi_trip_df['startstationname'], title='Top 10 starting station')
plt.tight_layout
plt.show()


In [ ]:

#~ counts of top 10 ending stations

top10_end_count = clean_bixi_trip_df['endstationname'].value_counts().head(10).sort_values()

top10_end_count.plot(kind='barh', xlabel='Count', ylabel=clean_bixi_trip_df['endstationname'], title='Top 10 ending station')
plt.tight_layout
plt.show()

In [ ]:

#~ counts of top 10 starting boroughs

top10_startBor_count = clean_bixi_trip_df['startstationarrondissement'].value_counts().head(10).sort_values()

top10_startBor_count.plot(kind='barh', xlabel='Count', ylabel=clean_bixi_trip_df['startstationarrondissement'], title='Top 10 starting boroughs')
plt.tight_layout
plt.show()

In [ ]:

#~ counts of top 10 ending boroughs

top10_endBor_count = clean_bixi_trip_df['endstationarrondissement'].value_counts().head(10).sort_values()

top10_endBor_count.plot(kind='barh', xlabel='Count', ylabel=clean_bixi_trip_df['endstationarrondissement'], title='Top 10 ending boroughs')
plt.tight_layout
plt.show()

In [ ]:



#~ bar chart to discover most popular day of week of trips for starting station

daily_counts_start = (clean_bixi_trip_df['start_day'].value_counts().sort_index())

ax = daily_counts_start.plot.bar(figsize=(12,4), rot=90)
ax.set_xlabel("Date of week")
ax.set_ylabel("Trips")
ax.set_title("Trips per day of week for starting stations")
plt.show()

In [ ]:

#~ bar chart to discover most popular day of week of trips for ending station

daily_counts_end = (clean_bixi_trip_df['end_day'].value_counts().sort_index())

ax = daily_counts_end.plot.bar(figsize=(12,4), rot=90)
ax.set_xlabel("Date of week")
ax.set_ylabel("Trips")
ax.set_title("Trips per day for ending station")
plt.show()

In [ ]:

#~ looking at the trips for each hour for starting station

hourly_counts_start = (clean_bixi_trip_df['start_hour'].value_counts().sort_index())

ax = hourly_counts_start.plot.bar(figsize=(12,4), rot=90)
ax.set_xlabel("Hour of day")
ax.set_ylabel("Trips")
ax.set_title("Trips per hour of day for starting station")
plt.show()

In [ ]:

#~ looking at the trips for each hour for starting station

hourly_counts_end = (clean_bixi_trip_df['end_hour'].value_counts().sort_index())

ax = hourly_counts_end.plot.bar(figsize=(12,4), rot=90)
ax.set_xlabel("Hour of day")
ax.set_ylabel("Trips")
ax.set_title("Trips per hour of day for ending station")
plt.show()

In [ ]:

#~ bar chart to discover most popular day of month of trips for starting station

daily_counts_startday = (clean_bixi_trip_df['start_dayofmonth'].value_counts().sort_index())

ax = daily_counts_startday.plot.bar(figsize=(12,4), rot=90)
ax.set_xlabel("Day of Month")
ax.set_ylabel("Trips")
ax.set_title("Trips per day of month for starting stations")
plt.show()

In [ ]:

#~ bar chart to discover most popular  month of trips for starting station

daily_counts_startmonth = (clean_bixi_trip_df['start_month'].value_counts().sort_index())

ax = daily_counts_startmonth.plot.bar(figsize=(12,4), rot=90)
ax.set_xlabel("Month")
ax.set_ylabel("Trips")
ax.set_title("Trips per month for starting stations")
plt.show()

In [ ]:

#~ filterting the bixi trip dataset to only have month 5,6,7,8, 9 and 10 

clean_bixi_trip_df.query('5 <= start_month <= 10', inplace=True)

clean_bixi_trip_df.reset_index(drop=True, inplace=True)

In [ ]:
clean_bixi_trip_df

## Weather Dataset

In [ ]:

#~ importing weather data set  source: https://montreal.weatherstats.ca/download.html 
#~ choose 'Climate Hourly' option and then press download

weather_df =  pd.read_csv('mtl_weather_2024.csv')  #! make sure your file name is same!
# weather_df =  pd.read_csv('mtl_weather_2024.csv') 
#~ checking top 5 rows of dataframe
weather_df.head() 



In [ ]:

#~ # Check the datatype of all columns
weather_df.info()

In [ ]:

# Convert unixtime to datetime (localized to America/Toronto time zone)
weather_df["datetime"] = pd.to_datetime(weather_df["unixtime"], unit="s", utc=True).dt.tz_convert("America/Toronto").dt.tz_localize(None)

# Create three new columns
weather_df["hour"] = weather_df["datetime"].dt.hour                  # hour of the day (0–23)
weather_df["day_of_week"] = weather_df["datetime"].dt.dayofweek + 1  # day of week (1–7, Monday=1)
weather_df["month"] = weather_df["datetime"].dt.month                # month of the year (1–12)
weather_df["year"] = weather_df['datetime'].dt.year                  # year
# Keep only data from May (5) to October (10)
updated_weather_df = weather_df[weather_df["month"].between(5, 10)]

#~ filtering only 2024
updated_weather_df = updated_weather_df.query('year == 2024').reset_index(drop=True)

updated_weather_df.head()

In [ ]:

# Quick sanity check
updated_weather_df.info()

In [ ]:

'''
Based on the missing value profiles and each variable’s potential impact on BIXI trips, 
we select temperature, wind_speed, relative_humidity, and visibility as the weather variables likely to affect ridership. 
And we treat conditions with high relative_humidity and low visibility as bad weather (e.g., rain).
Together with the newly created time variables, we build a new dataframe: updated_weather_df.
'''
weather_column_keep = ['datetime', 'hour', 'day_of_week', 'month',
                       'temperature', 'wind_speed', 'relative_humidity', 'visibility']    
to_use_weather_cols = [c for c in weather_column_keep if c in updated_weather_df.columns]
updated_weather_df = updated_weather_df[to_use_weather_cols].copy()

updated_weather_df

In [ ]:

# Check dtype of columns
# The data types seem to be fine as a baseline
updated_weather_df.dtypes

In [ ]:

updated_weather_df.isna().sum()

In [ ]:

# Fill in the missing values of visibility. Since visibility usually changes smoothly over time, linear interpolation is a suitable method
# Interpolate visibility linearly based on time
updated_weather_df["visibility"] = updated_weather_df["visibility"].interpolate(method="linear")

# If still any missing (e.g. at start/end of data), fill with nearest valid value
updated_weather_df["visibility"] = (
    updated_weather_df["visibility"].bfill().ffill())

updated_weather_df.isna().sum()

In [ ]:

#~  Distribution analysis of weather variables
import matplotlib.pyplot as plt

weather_cols = ["temperature", "wind_speed", "relative_humidity", "visibility"]

plt.figure(figsize=(12, 8))
for i, col in enumerate(weather_cols, 1):
    plt.subplot(2, 2, i)
    updated_weather_df[col].hist(bins=30, color='skyblue', edgecolor='black')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
plt.tight_layout()
plt.show()


**Interpretation:**
From the four histograms above, we can observe that the temperature distribution is approximately normal, with a reasonable range and no apparent outliers. 

Wind speed shows a right-skewed distribution, with a few instances of high values but still within a reasonable range. 

Relative humidity is mostly concentrated between 60% and 90%, with no outliers, indicating that Montreal generally experiences high humidity during summer and autumn, which suggests that BIXI ridership may be notably affected by rainy conditions.

For visibility, most values are concentrated around 24100 m, suggesting this is the upper limit of the measurement system. A few values near 50000 m far exceed realistic urban atmospheric visibility and are considered outliers. To address this, we categorized visibility into bins.

Based on meteorological conventions, when visibility is below 5000 m, conditions such as fog or rain are likely to cause a significant decline in ridership; when visibility is above 15000 m, weather conditions are generally stable and have minimal impact on travel. Therefore, visibility is divided into three categories — low, medium, and high — corresponding to the ranges 0–5000 m, 5000–15000 m, and above 15000 m, respectively.

In [ ]:

#~ Bin the visibility variable
#~ Define bins and labels
bins = [0, 5000, 15000, float('inf')] 
labels = ['low', 'medium', 'high']

#~ Create a new categorical column for visibility levels
updated_weather_df['visibility_level'] = pd.cut(
    updated_weather_df['visibility'],
    bins=bins,
    labels=labels,
    include_lowest=True,
    right=False 
)

# Check the distribution
print(updated_weather_df['visibility_level'].value_counts())


In [ ]:

import seaborn as sns

# Calculate the average relative humidity for each visibility category
grouped = updated_weather_df.groupby('visibility_level')['relative_humidity'].mean().reset_index()

# Visualize
sns.barplot(data=grouped, x='visibility_level', y='relative_humidity', color='skyblue')
plt.title('Average Relative Humidity by Visibility Level')
plt.xlabel('Visibility Level')
plt.ylabel('Average Relative Humidity (%)')
plt.show()

# Calculate the Spearman rank correlation between relative humidity and visibility level
corr = updated_weather_df['relative_humidity'].corr(updated_weather_df['visibility_level'].cat.codes, method='spearman')
print("Spearman correlation:", corr)

**Interpretation:**  As visibility decreases, average relative humidity rises sharply. Low-visibility periods correspond to humidity above 90%, while clear conditions show about 70%. The negative Spearman correlation (-0.315) confirms that high humidity is typically associated with reduced visibility, indicating foggy or rainy weather. This supports creating a bad_weather variable combining humidity and visibility to capture adverse cycling conditions.

Based on meteorological conventions, we define bad_weather as cases where relative_humidity > 85% and visibility < 10000 m. A humidity level above 85% generally indicates very moist conditions and often precedes visible precipitation, while visibility below 10,000 meters typically indicates foggy or rainy weather.

In [ ]:

# Create a new variable bad_weather
updated_weather_df["bad_weather"] = (
    (updated_weather_df["relative_humidity"] > 85) &
    (updated_weather_df["visibility"] < 10000)
).astype(int)

# Check the distribution of bad vs. good weather
print(updated_weather_df["bad_weather"].value_counts())

In [ ]:
'''
Considering that temperature and wind_speed may require standardization for certain models, we create two new columns to store their standardized values. 
At the same time, the original columns are retained in the final dataframe to allow future exploration with models that do not require standardization.
'''
from sklearn.preprocessing import StandardScaler

cols_to_scale = ['temperature', 'wind_speed']
scaler = StandardScaler()

# Fit and transform the selected columns
scaled_values = scaler.fit_transform(updated_weather_df[cols_to_scale])

# Create new standardized columns
for i, col in enumerate(cols_to_scale):
    updated_weather_df[f"{col}_scaled"] = scaled_values[:, i]

updated_weather_df.head()

In [ ]:

# Choose and reorder the columns of updated_weather_df and reset the index to create the final cleaned_weather_df
cleaned_weather_df = updated_weather_df[['datetime', 'hour', 'day_of_week', 'month',
                                        'temperature', 'temperature_scaled', 'wind_speed', 'wind_speed_scaled', 'bad_weather']]
cleaned_weather_df = cleaned_weather_df.reset_index(drop=True)
cleaned_weather_df

# Feature Engineering and EDA cycle (back and forth)

<h5 style='color:Orange'> As of this instance, <br>

*<strong> the 3 variable for our clean dataframe of our datasets are the following:* </strong> <br>


<strong> clean_stations_df </strong>  : the station info (capacity, name, id, etc.)

<strong> clean_bixi_trip_df </strong> : the bixi historical data of 2024 data

<strong> cleaned_weather_df </strong>: montreal 2024 weather data per day 



</h5>

## Feature Engineering for Bixi_trips

In [ ]:
clean_bixi_trip_df

In [ ]:

#~ to find the duration of each trip. 

clean_bixi_trip_df['trip_duration'] = clean_bixi_trip_df['endtime'] - clean_bixi_trip_df['starttime']

clean_bixi_trip_df

In [ ]:

#~ converting the trip_duration in minutes

td = pd.to_timedelta(clean_bixi_trip_df['trip_duration'], errors='coerce')


#~ rounding the duration to the nearest minute
clean_bixi_trip_df['trip_duration_min'] = td.dt.total_seconds() / 60
clean_bixi_trip_df['trip_duration_min_round'] = clean_bixi_trip_df['trip_duration_min'].round(2)

#~ dropping columns that won't be useful for our model
clean_bixi_trip_df = clean_bixi_trip_df.drop(columns=['trip_duration_min','startstationlatitude','startstationlongitude','endstationlatitude','endstationlongitude'])

clean_bixi_trip_df




In [ ]:

#~ having a quick descriptive statistic for our trip duration 
with pd.option_context('display.float_format', '{:,.2f}'.format):
    print(clean_bixi_trip_df['trip_duration_min_round'].describe())

In [ ]:

#~ it seems like there are trip duration <= 0 min, therefore dropping all of them
#~ in this instance we keep only tripduration > 0 

clean_bixi_trip_df = (clean_bixi_trip_df[clean_bixi_trip_df['trip_duration_min_round'] > 0].reset_index(drop=True))

clean_bixi_trip_df

In [ ]:

#~ Looking at the descriptive stats of the column, it seems there are outliers
#~ building a boxplot to see how severe it is. 

s = pd.to_numeric(clean_bixi_trip_df["trip_duration_min_round"], errors="coerce").dropna()

sns.set_theme(style="whitegrid")
plt.figure()
sns.boxplot(x=s)
plt.xlabel("Trip duration (minutes)")
plt.title("BIXI Trip Duration")
plt.show()

In [ ]:


"""Using Z-Score to identify outliers"""

col = "trip_duration_min_round"

s = pd.to_numeric(clean_bixi_trip_df[col], errors="coerce")

#~ compute Z-score
mu, sigma = s.mean(skipna=True), s.std(skipna=True, ddof=0)
z = (s - mu) / sigma

#~ creating a column to flag if it is a outlier Z > 3
thr = 3.0
clean_bixi_trip_df["z_trip_min"] = z
clean_bixi_trip_df["is_outlier_z3"] = z.abs() > thr

#~quick summary
n = s.notna().sum()
n_out = clean_bixi_trip_df["is_outlier_z3"].sum()
print(f"Outliers (|z|>{thr}): {n_out} of {n} = {n_out/n:.2%}")

#~ to view outliers
outliers = clean_bixi_trip_df.loc[clean_bixi_trip_df["is_outlier_z3"]].sort_values(col, ascending=False)

outliers


In [ ]:

#~ dropping all rows that have been tagged as outliers (keep only non-outliers)

clean_bixi_trip_df = (
    clean_bixi_trip_df.loc[~clean_bixi_trip_df["is_outlier_z3"]]
    .reset_index(drop=True)
)

In [ ]:

#~ quick check of remaining data,  it seems there are still outliers.. 1000 min of trip duration...

with pd.option_context('display.float_format', '{:,.2f}'.format):
    print(clean_bixi_trip_df['trip_duration_min_round'].describe())

In [ ]:

''' After a bit of research, BIXI charges 18 cents after 45 minutes of the initial use. This encourage people to dock their bikes
for a break to restart the 45-minutes free period. With this in mind, we put a hard cutoff for any trip duration that is more than 120 min  '''

clean_bixi_trip_df = (
    clean_bixi_trip_df[clean_bixi_trip_df["trip_duration_min_round"] <= 120]
    .reset_index(drop=True)
)


In [ ]:

#~ dropping the outlier column tag created earlier
clean_bixi_trip_df = clean_bixi_trip_df.drop(columns=['is_outlier_z3'])

In [ ]:
sub = s[(s > 0) & (s <= 120)].dropna()

sns.set_theme(style="whitegrid")
plt.figure()
sns.histplot(sub, bins=range(0, 121, 5), edgecolor="white")  
plt.xlim(0, 120)
plt.xlabel("Trip duration (minutes)")
plt.ylabel("Count")
plt.title("BIXI Trip Duration (<= 120 min)")
plt.show()

In [ ]:

'''  The dataset for bixi trip is now updated to have duration'''
clean_bixi_trip_df


# Towards the final dataset for modelling


<h5>

clean data set names: 

clean_stations_df  ===> station dataset

clean_bixi_trip_df ===> bixi 2024 trip history

cleaned_weather_df ===> hourly weather dataset from 2024

</h5>

In [ ]:
clean_bixi_trip_df

In [ ]:
cleaned_weather_df

We create the final dataset ready for modeling model_df by filtering the top400 stations with more demand overall and grouping those stations by demand per hour (sum of start trip + end trip). Then merge with weather dataset.

In [ ]:
#  hourly rows for STARTS (departures) and ENDS (arrivals)
starts = (
    clean_bixi_trip_df
      .loc[:, ['startstationname', 'startstationarrondissement', 'starttime']]
      .rename(columns={
          'startstationname': 'station',
          'startstationarrondissement': 'arrondissement',
          'starttime': 'datetime'
      })
)
ends = (
    clean_bixi_trip_df
      .loc[:, ['endstationname', 'endstationarrondissement', 'endtime']]
      .rename(columns={
          'endstationname': 'station',
          'endstationarrondissement': 'arrondissement',
          'endtime': 'datetime'
      })
)

# Hourly buckets (so starts/ends land in their own hour)
starts['datetime'] = starts['datetime'].dt.floor('h')
ends['datetime']   = ends['datetime'].dt.floor('h')

#  concatenate starts and ends and count each row = 1 unit of demand
long_df = pd.concat([starts, ends], ignore_index=True)
long_df['ones'] = 1

# keep only the TOP 400 stations by total demand OVERALL
# total demand per station (starts + ends) from the long table
station_totals = (
    long_df.groupby('station', as_index=False)['ones']
           .sum()
           .sort_values('ones', ascending=False)
)

top_400 = station_totals.head(400)['station']

# Filter long_df to just those stations (TOP 400)
long_df = long_df[long_df['station'].isin(top_400)].copy()


# Hourly demand per station + arrondissement ( both starts & ends combined)
hourly_station_demand = (
    long_df
      .groupby(['station', 'arrondissement', 'datetime'], as_index=False)['ones']
      .sum()
      .rename(columns={'ones': 'total_demand'})
)

hourly_station_demand['hour']  = hourly_station_demand['datetime'].dt.hour
hourly_station_demand['day']   = hourly_station_demand['datetime'].dt.day
hourly_station_demand['month'] = hourly_station_demand['datetime'].dt.month
hourly_station_demand['year']  = hourly_station_demand['datetime'].dt.year

# Merge with weather (per hour) 
weather_cols = ['datetime', 'temperature', 'wind_speed', 'bad_weather',
                'temperature_scaled', 'wind_speed_scaled', 'day_of_week', 'month', 'hour']
weather = cleaned_weather_df.loc[:, [c for c in weather_cols if c in cleaned_weather_df.columns]]

model_df = hourly_station_demand.merge(
    weather, on='datetime', how='left'
)

From the weather data there's some values from November 1, that we don't have in the bixi_trips dataset, we remove those:

In [ ]:
model_df.isna().sum()

In [ ]:
model_df[model_df['temperature'].isna()]

In [ ]:
#Remove NAN values, from dates that aren't on Bixi_trips (november 1)
model_df = model_df.dropna(subset=['temperature', 'wind_speed', 'temperature_scaled', 'wind_speed_scaled'])
model_df = model_df.reset_index(drop=True)

In [ ]:
model_df

In [ ]:
# drop duplicate columns and rename
model_df = (
    model_df
      .drop(columns=['hour_y','month_y'], errors='ignore')
      .rename(columns={'hour_x':'hour','month_x':'month'})
      .reset_index(drop=True)
)

In [ ]:
# Move total_demand to the end of the dataframe
cols = [c for c in model_df.columns if c != 'total_demand'] + ['total_demand']
model_df = model_df[cols]

In [ ]:
#FINAL DATASET READY FOR MODELING
model_df

In [ ]:

#~ adding new features =====================================================

#~ is weekend?
model_df['is_weekend'] = model_df['day_of_week'].isin([5,6]).astype(int)



# #~ cycle time loop 
model_df['hour_sin'] = np.sin(2*np.pi*model_df['hour']/24)
model_df['hour_cos'] = np.cos(2*np.pi*model_df['hour']/24)



#~ feels like weather

model_df['feels_like'] = model_df['temperature'] - 0.7 * model_df['wind_speed']


# # #~ borought encoded:

# model_df['arrondissement_encoded'] = model_df['arrondissement'].astype('category').cat.codes

#~ avg_hourly_demand_station
model_df = model_df.sort_values(['station','datetime'])

model_df['avg_hourly_demand_station'] = (
    model_df.groupby(['station','hour'])['total_demand']
      .apply(lambda s: s.shift(1).expanding(min_periods=1).mean())
      .reset_index(level=[0,1], drop=True)
)
#~ average day of week station

avg_dow_hist = (
    model_df.groupby(['station','day_of_week'])['total_demand']
            .apply(lambda s: s.shift(1).expanding(min_periods=1).mean())
            .reset_index(level=[0,1], drop=True)
)

model_df['avg_dayofweek_station'] = avg_dow_hist

#~ for stations with no historical 
station_past_mean = (
    model_df.groupby('station')['total_demand']
            .apply(lambda s: s.shift(1).expanding(min_periods=1).mean())
            .reset_index(level=0, drop=True)
)

for col in ['avg_hourly_demand_station', 'avg_dayofweek_station']:
    model_df[col] = model_df[col].fillna(station_past_mean)

#~ fill station without historical data
model_df['avg_dayofweek_station'] = model_df['avg_dayofweek_station'].fillna(0)

#~ fill hourly demand station without historical data
model_df['avg_hourly_demand_station'] = model_df['avg_hourly_demand_station'].fillna(0)


#~ adding holiday Quebec/Montreal public holidays 
holidays_2024 = [
    "2024-01-01",  # New Year's Day
    "2024-03-29",  # Good Friday
    "2024-04-01",  # Easter Monday (public sector)
    "2024-05-20",  # National Patriots' Day
    "2024-06-24",  # Fête nationale du Québec
    "2024-07-01",  # Canada Day
    "2024-09-02",  # Labour Day
    "2024-10-14",  # Thanksgiving
    "2024-12-25",  # Christmas
    "2024-12-26",  # Boxing Day
]

holidays_2024 = pd.to_datetime(holidays_2024)

#~ adding if holiday in df 

model_df['is_holiday'] = model_df['datetime'].dt.date.isin(holidays_2024.date).astype(int)

#~ weather interaction terms (temperature x hour)

model_df['temp_hour']= model_df['temperature_scaled']*model_df['hour']

#~ curved comfort pattern of temperature

model_df['temperature_sq'] = model_df['temperature_scaled']**2 

#~ temperature x feels_like interaction term: 

model_df['temp_feels_interaction'] = model_df['temperature_scaled'] * model_df['feels_like']

#~ simplyifing daily patterns night, morning, day, evening, late

model_df['hour_bucket'] = pd.cut(model_df['hour'],
                           bins=[-1,5,10,16,19,24],
                           labels=['night','morning','day','evening','late'],
                           include_lowest=True).astype('category').cat.codes

#~ weekend hour interaction 

model_df['weekend_hour_interaction'] = model_df['is_weekend'] * model_df['hour_sin']




In [ ]:

#~ creating a small dataframe for joining the station names to lat and lon
coords_df = (
    bixi_trip_df[
        ['startstationname', 'startstationlatitude', 'startstationlongitude']
    ]
    .drop_duplicates(subset='startstationname')  # keep 1 row per station
    .rename(columns={
        'startstationname': 'station',
        'startstationlatitude': 'lat',
        'startstationlongitude': 'lon'
    })
)


In [ ]:
model_df = model_df.merge(
    coords_df[['station', 'lat', 'lon']],
    on='station',
    how='left'
)

#~ bixi_trip_df

In [ ]:
model_df

In [ ]:

#~ exporting csv 

model_df.to_csv("BIXI_MODEL.csv", index=False)